In [1]:
!nvidia-smi

Sat Mar 27 08:38:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install feedback-transformer-pytorch
!pip install transformers
!pip install torchtext

     |████████████████████████████████| 2.0MB 5.7MB/s 
     |████████████████████████████████| 3.2MB 47.6MB/s 
     |████████████████████████████████| 890kB 43.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=932d383d1f10c76804c22ed43294da64aad84dd713641d0371f7852cc780f9e4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/MyDrive'
!ls

/content/drive/MyDrive
 Data  'FeedBack Transformer'   Tokenizer_26_03


In [5]:
import torch
from transformers import AutoModel, AutoTokenizer
from feedback_transformer_pytorch import FeedbackTransformer
from transformers import RobertaTokenizerFast
import os
import torch
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from filelock import FileLock
from transformers.utils import logging
from typing import Dict, List, Optional
import pickle
import random
import time
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import numpy as np

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = RobertaTokenizerFast.from_pretrained("./Tokenizer_26_03", max_len=512)
tokenizer.add_tokens('\n')
vocab_size= tokenizer.vocab_size
vocab_size = vocab_size + 1

In [7]:
vocab_size

11983

In [8]:
train_path = 'Data/Poem/train_dataset_24_03.txt'
test_path = 'Data/Poem/valid_dataset_24_03.txt'

In [ ]:
# with open(train_path, encoding="utf-8") as f:
#     total_poem = f.read()
# split_total_poem = total_poem.split("\n\n")
# canto_poem = [split_total_poem[x:x+4] for x in range(0, len(split_total_poem), 4)]
# canto_poem = ["\n\n".join(i) for i in canto_poem]

# canto_token = [tokenizer.encode(i) for i in canto_poem]
# canto_token = [i for i in canto_token if len(i) >= 129 and len(i) <= 140]
# len(canto_token)

35969

In [ ]:
# print(add_padding(canto_token[0],140))

[0, 1048, 871, 471, 917, 1129, 747, 203, 4588, 1642, 871, 1156, 1348, 982, 858, 569, 203, 7839, 825, 4863, 9947, 2453, 426, 203, 5491, 5346, 3578, 3251, 730, 4676, 11680, 2067, 203, 203, 758, 2453, 4703, 665, 363, 1787, 203, 1093, 471, 2453, 1642, 1584, 370, 737, 1471, 203, 2707, 1114, 551, 496, 388, 3407, 203, 5125, 3119, 6580, 587, 904, 6255, 440, 959, 203, 203, 4588, 566, 7746, 1417, 5182, 5761, 203, 2626, 2457, 440, 4136, 871, 5182, 871, 5394, 203, 3384, 2453, 838, 952, 509, 2729, 203, 1093, 4823, 871, 8299, 818, 5099, 424, 846, 203, 203, 1096, 689, 871, 8624, 2453, 1186, 203, 4265, 1156, 7703, 3741, 1573, 872, 632, 781, 203, 1212, 424, 2453, 4653, 871, 4528, 203, 2011, 977, 1372, 1049, 432, 1544, 509, 1350, 2, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# test_len = []
# for i in canto_poem:
#   test_len.append(len(tokenizer.encode(i)))
# from collections import Counter
# print(Counter(test_len))

In [9]:
def add_padding(list_token: list, block_size:int):
    tmp_list = [0]* block_size
    tmp_list[0:len(list_token)] = list_token
    tmp_list[len(list_token):block_size] = [1]*(block_size-len(list_token))
    return tmp_list

In [10]:
logger = logging.get_logger(__name__)
class CusTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        # num_special_tokens_to_add Returns the number of added tokens when encoding a sequence with special tokens

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")

                self.examples = []
                with open(file_path, encoding="utf-8") as f:
                    total_poem = f.read()

                # tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

                # for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                # i = 0
                # while i < len(tokenized_text) - block_size + 1:
                #     inds = tokenized_text[i : i + block_size]
                #     for j in range(0, len(inds)):
                #         if inds[j]==64001:
                #             inds = inds[j+1:] #remove the first \n
                #             break
                #     for j in range(len(inds)-1, 0, -1):
                #         if inds[j]==64001:
                #             inds = inds[:j-1] #remove \n
                #             break
                #     i += len(inds)
                #     self.examples.append(
                #         tokenizer.build_inputs_with_special_tokens(inds)
                #     )
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should loook for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.
                split_total_poem = total_poem.split("\n\n")
                canto_poem = [split_total_poem[x:x+4] for x in range(0, len(split_total_poem), 4)]
                canto_poem = ["\n\n".join(i) for i in canto_poem]

                canto_token = [tokenizer.encode(i) for i in canto_poem]
                print(canto_token)
                canto_token = [i for i in canto_token if len(i) >= 129 and len(i) <= 140]

                for i in canto_token:
                  self.examples.append(add_padding(i,block_size=block_size ))


                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [11]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling, LineByLineWithSOPTextDataset

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = CusTextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=140)
     
    test_dataset = CusTextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=140)   
    
    return train_dataset,test_dataset

train_dataset,test_dataset = load_dataset(train_path,test_path, tokenizer)

In [12]:
len(train_dataset[0])

140

In [13]:
print(train_dataset[0])

tensor([    0,  1048,   871,   471,   917,  1129,   747, 11982,  4588,  1642,
          871,  1156,  1348,   982,   858,   569, 11982,  7839,   825,  4863,
         9947,  2453,   426, 11982,  5491,  5346,  3578,  3251,   730,  4676,
        11680,  2067, 11982, 11982,   758,  2453,  4703,   665,   363,  1787,
        11982,  1093,   471,  2453,  1642,  1584,   370,   737,  1471, 11982,
         2707,  1114,   551,   496,   388,  3407, 11982,  5125,  3119,  6580,
          587,   904,  6255,   440,   959, 11982, 11982,  4588,   566,  7746,
         1417,  5182,  5761, 11982,  2626,  2457,   440,  4136,   871,  5182,
          871,  5394, 11982,  3384,  2453,   838,   952,   509,  2729, 11982,
         1093,  4823,   871,  8299,   818,  5099,   424,   846, 11982, 11982,
         1096,   689,   871,  8624,  2453,  1186, 11982,  4265,  1156,  7703,
         3741,  1573,   872,   632,   781, 11982,  1212,   424,  2453,  4653,
          871,  4528, 11982,  2011,   977,  1372,  1049,   432, 

In [14]:
print(tokenizer.decode(train_dataset[0]))

<s>lời ông sao quá mặn nồng
tui sợ ông bồng đem thả xuống sông
thoai đừng dụ khị tui không
mắc bẫy phập phồng mấy đợt ngoán ghê

ý tui hỏng phải là chê
nhưng sao tui sợ vỗ về ngọt ngon
sau lưng thì cả một đòn
tuyệt chiêu khuyến mãi đang lòn trong kia

tui đây ớn cá lia thia
nằm sẵn trong chậu ông lia ông dằn
tính tui lắm lúc cũng căng
nhưng đụng ông chằn nên lủi cho yên

giờ nghe ông noái tui phiền
ẵm bồng roài liệng ngả nghiêng thân già
thôi cho tui miễn ông ha
chồng vợ đậm đà như vậy cũng xong</s><pad><pad><pad><pad><pad><pad><pad><pad>


#Initialize Model

In [15]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset= test_dataset, batch_size= 32, shuffle=True)

In [19]:
count = 0
for i in train_loader:
  count = count +1 

print(count)

1125


In [16]:
import torch
from feedback_transformer_pytorch import FeedbackTransformer

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

model = FeedbackTransformer(
    num_tokens = vocab_size,           # number of tokens
    dim = 512,                    # dimension
    depth = 6,                    # depth
    seq_len = 8,                  # the sequence length of each segment or window
    mem_len = 256,                # length of the memory buffer
    dim_head = 64,                # dimension of each head
    heads = 8,                    # number of heads
    attn_dropout = 0.1,           # attention dropout
    ff_dropout = 0.1              # feedforward dropout
).to(device)

In [18]:
x = torch.randint(0, vocab_size, (32, 140)).cuda()
print(model(x).shape)

torch.Size([32, 140, 11983])


#Train Feedback Transformer


In [20]:
lr_rate = 0.001
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr_rate)
nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id).to(device)


CrossEntropyLoss()

In [21]:
def save_checkpoint(state, filename= "FeedBack Transformer/feedback_transformer_checkpoint_update.pth.tar"):
    print("Saving checkpoint")
    torch.save(state,filename)

def load_checkpoint(state):
    print("Load checkpoint")
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])


In [ ]:
# load_checkpoint(torch.load("FeedBack_Transformer/feedback_transformer_checkpoint_update.pth.tar"))

Load checkpoint


In [ ]:
 def train(model, train_loader, device, criterion, vocab_size, optimizer, num_epochs):
    losses = 0
    n_batches, n_samples = len(train_loader), len(train_loader.dataset)
    
    model.train()
    torch.autograd.set_detect_anomaly(True)
    for index_epoch in range(num_epochs):
      print('='*20)
      print('epoch: {}'.format(index_epoch))

      checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
      save_checkpoint(checkpoint)
      print('Loss: {}'.format(losses))
      for i, batch in enumerate(train_loader):
        inputs = batch.to(device)
        targets = inputs[:, 1:].contiguous()
        
        lm_logits = model(inputs,return_memory = False)
        lm_logits = lm_logits[:, :-1].contiguous()
        
        loss = criterion(lm_logits.view(-1, vocab_size), targets.view(-1))
        print('index: {}, batch: {}, loss: {}'.format(i, batch, loss))
        losses += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
      
train(model, train_loader, device, criterion, vocab_size, optimizer,num_epochs= 100)

Streaming output truncated to the last 5000 lines.
        [   0, 1576,  835,  ...,    1,    1,    1],
        [   0,  773, 2971,  ...,    1,    1,    1]]), loss: 3.0906248092651367
index: 219, batch: tensor([[   0, 4668, 1015,  ...,    1,    1,    1],
        [   0, 2568,  523,  ...,    1,    1,    1],
        [   0, 1324,  534,  ...,    1,    1,    1],
        ...,
        [   0, 3074,  534,  ...,    1,    1,    1],
        [   0,  589, 1444,  ...,    1,    1,    1],
        [   0, 1964,  435,  ...,    1,    1,    1]]), loss: 3.199340581893921
index: 220, batch: tensor([[   0, 1075,  471,  ...,    1,    1,    1],
        [   0, 2072,  365,  ...,    1,    1,    1],
        [   0,  638,  345,  ...,    1,    1,    1],
        ...,
        [   0, 3405,  610,  ...,    1,    1,    1],
        [   0, 2486, 1615,  ...,    1,    1,    1],
        [   0, 3312, 2202,  ...,    1,    1,    1]]), loss: 3.147766590118408
index: 221, batch: tensor([[    0,   589,  1444,  ...,     1,     1,     1],
 

In [ ]:
class TextGenerator():

    def __init__(self, max_tokens, start_tokens, maxlen, model, tokenizer,device, topk):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.maxlen = maxlen
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.k = topk 

    def sample_from(self, logits):
        logits, indices = torch.topk(logits, k=self.k, sorted=True)
        return np.random.choice(indices.cpu().numpy())


    def gen_poem(self):
        start_tokens = [_ for _ in self.start_tokens]
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = self.maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:self.maxlen]
                sample_index = self.maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = torch.tensor([x], device= self.device)
            y = self.model(x)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
            # print(sample_token)
        output_token = [_ for _ in self.start_tokens + tokens_generated]
        poem = self.tokenizer.decode(output_token)
        print(f"generated text:\n{poem}\n")

In [ ]:
num_token_generated = 50
hint = 'mùa xuân'
start_tokens = tokenizer.encode(hint)[:-1]
generator = TextGenerator(max_tokens= num_token_generated, start_tokens= start_tokens, maxlen= 300, model= model, tokenizer= tokenizer,device= device, topk= 1)
generator.gen_poem()

generated text:
<s>mùa xuân chim nó thật thà 
ru anh một khúc dân ca một mình 
mặt anh bơi lặn nhiều đêm 
cho em giải thoát khỏi niềm vì em 
cho anh cây sống dài thêm 
cho em có hi thèm kim đáy sông 



In [ ]:
num_token_generated = 30
hint = 'mùa thu'
start_tokens = tokenizer.encode(hint)[:-1]
generator = TextGenerator(max_tokens= num_token_generated, start_tokens= start_tokens, maxlen= 300, model= model, tokenizer= tokenizer,device= device, topk= 2)
generator.gen_poem()

generated text:
<s>mùa thu hà nội hoa khe
núi già bóng phù sa ngắn dài nhìn theo tà nhỏ cuộc đời bên hiên sau

em mang chiếc bóng chơi vơi  thui

